In [6]:
import os
import re
import sys
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import skimage
from skimage import io
from sklearn import preprocessing
from tqdm.notebook import tqdm, trange
import anndata as ad
import cv2
import pickle

In [7]:
p_dir = (Path().cwd().parents[0].parents[0]).absolute()
data_dir = p_dir / "09_datasets"

In [8]:
mask_folder = data_dir / 'exported' / 'foll_masks'

# Functions

In [11]:
# def mask2boundary(mask):
#     contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     contour = max(contours, key = cv2.contourArea)
#     contour = np.squeeze(contour)
#     contour[:, [0,1]] = contour[:, [1,0]]
#     return contour

def mask2boundary(mask):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour = np.empty(2)
    for i in range(len(contours[0])):
        contour = np.vstack((contours[0][i][0], contour))
    contour = contour[0:-1]
    contour.T[0] = contour.T[0] + 1
    contour.T[1] = contour.T[1] + 1
    boundary = np.empty((2, len(contour.T[1])))
    boundary[0] = contour.T[1]
    boundary[1] = contour.T[0]
    boundary = boundary.T.astype(int)
    return boundary

In [12]:
import PIL
from collections import defaultdict

info = defaultdict(list)

# Loop through image folder
for (dirpath, dirnames, filenames) in os.walk(mask_folder):
    for name in sorted(filenames):
        if "png" not in name or 'foll' not in name:
            continue
            
        n_split = name.split('_')
        if len(n_split)>2:
            dataset = n_split[0]
            subset = '_' + n_split[1]
        else:
            dataset = n_split[0]
            subset = ''
        
        path_foll = os.path.join(dirpath, name)
        path_GC = os.path.join(dirpath, dataset + subset + '_GC.png')
        
        info['Path_foll'].append(path_foll)
        info['Path_GC'].append(path_GC)
        info['Dataset'].append(dataset)
        info['Subset'].append(subset)

In [13]:
df_info = pd.DataFrame(info)
df_info = df_info[df_info.Dataset == '00']